In [ ]:
import torch
from torchvision import models
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import time
import tqdm
from spectral import envi
import os
import random
import pandas as pd
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
import importlib
from PIL import Image
import matplotlib
import pickle
from sklearn.preprocessing import MinMaxScaler

In [ ]:
os.chdir(r'<YOUR_PATH_HERE>')
import conv_autoencoder
importlib.reload(conv_autoencoder)

## Load data (Choose to normalize or not the data)

In [ ]:
data_train=np.load(r'<YOUR_PATH_HERE>')
scaler1=MinMaxScaler()
data_train=scaler1.fit_transform(data_train.T).T
data_test=np.load(r'<YOUR_PATH_HERE>')
scaler=MinMaxScaler()
data_test=scaler.fit_transform(data_test.T).T
data_slice=np.load(r'<YOUR_PATH_HERE>')
scaler=MinMaxScaler()
data_slice=scaler.fit_transform(data_slice.T).T

In [ ]:
train_dataloader = DataLoader(np.expand_dims(data_train,1), batch_size=1024, shuffle=True)
valid_dataloader = DataLoader(np.expand_dims(data_test,1), batch_size=512, shuffle=False)

# Train

### Create basis matrix that initialized with mean spectrum and repeated same number of times as endmember number

In [ ]:
nb_endmember = 5
nb_bands = 230
basis=np.repeat(data_train.mean(axis=0).reshape(-1,1),nb_endmember,axis=1)

In [ ]:
importlib.reload(conv_autoencoder)
device = torch.device("cuda:0")
model=conv_autoencoder.ConvAutoencoder(nb_bands, torch.from_numpy(basis).float(),nb_endmember).to(device)
model.weight.requires_grad_grad=True # Here we can freeze the weights which are the endmember to force them not to change during training
list1=[]
list2=[]
for name, param in model.named_parameters():
    if name in ['weight']:
        list1.append(param)
    else:
        list2.append(param)
optimizer =torch.optim.Adam([{'params':list1,'lr':1e-6},{'params':list2,'lr':1e-4}])
path=r'<YOUR_PATH_HERE>'
name= f"model_{nb_endmember}_endmembers"
train_loss,valid_loss=conv_autoencoder.train_nn(200,model,optimizer,train_dataloader,valid_dataloader,0,0,path,name,device,True)

# Plot Endmembers

In [ ]:
hdr=envi.open(r'<YOUR_HDR_PATH_HERE>')
bands=np.array(hdr.bands.centers)
noise1=(bands<1136)
noise2=(bands>2413)
combined=noise1 | noise2 
model=conv_autoencoder.ConvAutoencoder(nb_bands, torch.from_numpy(np.random.rand(nb_bands,nb_endmember)).float(),nb_endmember).to(device)
path=r'<YOUR_MODEL_PATH_HERE>'
name= f"model_{nb_endmember}_endmembers"
model.load_state_dict(torch.load(os.path.join(path,name)))
cmap = plt.get_cmap('Set1')
colors = cmap(np.linspace(0, 1, model.weight.T.shape[1]+1))
colors=['orange', 'red','aqua','blue','green']
fig=plt.figure(dpi=200)
for i in range(model.weight.T.shape[1]):
    plt.plot(bands[~combined],model.weight.T[:,i].detach().cpu(),color=colors[i])
plt.legend(labels=[str(i) for i in range(model.weight.T.shape[1])])
plt.xlabel('# Bands')
plt.ylabel('Normalized Reflectance')
plt.title('Endmembers')
plt.close(fig)

## Abundances


In [ ]:
image = np.load(r"<YOUR_PATH_HERE>")
image,mask=conv_autoencoder.clean_image(image,bands,0.1,0.9)
scaler=MinMaxScaler()
image=scaler.fit_transform(image.reshape(-1,230).T).T
valid_dataloader = DataLoader(np.expand_dims(image.reshape(-1,230),1), batch_size=512, shuffle=False)
outputs=conv_autoencoder.get_abundances(model,valid_dataloader,device,True)
             